# Todo el proceso con el algoritmo con ceros

En este notebook se va a realizar todo el proceso hecho hasta ahora pero usando otra versión del algoritmo. 

Esta nueva versión consiste en rellenar las dimensiones con 0 hasta que el tamaño de la dimensión sea multiplo del tamaño de la partición. 

De esta forma se simplifica mucho el código con una consequente bajada del tiempo de ejecución.

## Paquetes necesarios

In [1]:
import transformData2 as td
import matplotlib.pyplot as plt
import numpy as np
import pycblosc2 as cb2
import time as t
import tables as tb

## Definición del conjunto de datos

In [2]:
def file_reader(filename, dataset):
    with tb.open_file(filename) as f:
        child = f.root
        for element in dataset.split('/'):
            child = child.__getattr__(element)
        return child[:]

### Datos sin transformación

In [3]:
PATH = '/home/aleix/datasets/'

filename = 'WRF_India.h5'

dataset = 'U'

src = file_reader(PATH + filename, dataset)

SHAPE = src.shape

SIZE = np.prod(SHAPE)

ITEMSIZE = src.dtype.itemsize

### Datos transformados

In [4]:
start = t.perf_counter()

TSUBSHAPE = [64, 4, 4, 32]

srct, TSHAPE = td.tData(src, TSUBSHAPE, inverse=False)

indexation = td.createIndexation(TSHAPE, TSUBSHAPE)

end = t.perf_counter()

ttran_tra = end - start

## Compresión y descompresión de los datos

In [5]:
KB = 1024
MB = 1024 * KB
GB = 1024 * MB

BLOSC_MAX_FILTERS = 5
BLOSC_BLOCKSIZE = 16 * KB
BLOSC_TYPESIZE = 4
BLOSC_CODE = 5

cparams = cb2.blosc2_create_cparams(compcode=BLOSC_CODE, clevel=8, use_dict=0, typesize=BLOSC_TYPESIZE,
                                        nthreads=4, blocksize= BLOSC_BLOCKSIZE, schunk=None,
                                        filters=[0, 0, 0, 0, 1],
                                        filters_meta=[0, 0, 0, 0, 0])

dparams = cb2.blosc2_create_dparams(nthreads=4, schunk=None)

### Datos sin transformación

#### Compresión

In [6]:
start = t.perf_counter()

schunk = td.compress(cparams, dparams, src)

end = t.perf_counter()

torig_com = end - start

#### Descompresión

In [7]:
start = t.perf_counter()

res = td.decompress(schunk, ITEMSIZE, SHAPE, c=17, d=231)

end = t.perf_counter()

torig_dec = end - start

### Datos transformados

#### Compresión

In [8]:
start = t.perf_counter()

schunk_t = td.compress_trans(cparams, dparams, srct, TSHAPE, TSUBSHAPE)

end = t.perf_counter()

ttran_com = end - start

#### Descompresión

In [9]:
start = t.perf_counter()

res2 = td.decompress_trans(schunk_t, indexation, ITEMSIZE, SHAPE, TSHAPE, TSUBSHAPE,  c=17, d=231)

end = t.perf_counter()

ttran_dec = end - start

## Resultados

In [10]:
print('La compresión de los datos transformados es {:.3f}x más rápida que la original.'.format(torig_com/(ttran_com + ttran_tra)))

print('La descompresión de los datos transformados es {:.3f}x más rápida que la original.'.format(torig_dec/ttran_dec))


La compresión de los datos transformados es 0.126x más rápida que la original.
La descompresión de los datos transformados es 18.252x más rápida que la original.


In [11]:
np.testing.assert_array_equal(res, res2)